# How Does Actions Work?

#### The workflow object is the key piece here. A GitHub Actions workflow is a set of code that defines a sequence and set of steps to execute, similar to a script or a program. The file itself must be coded in YAML format and stored in the <repository>/.github/workflows directory.

#### Workflow files have a specific syntax. A workflow contains one or more jobs. Each job can be as simple or as complex as needed. Once a workflow is kicked off, the jobs begin executing. By default, they run in parallel.

#### Jobs, in turn, are made up of steps. A step either runs a shell command or invokes a predefined GitHub action. All of the steps in a job are executed on a runner. The runner is a server (virtual or physical) or a container that has been set up to understand how to interact with GitHub Actions.

#### In GitHub Actions, the changes that signal that work needs to be kicked off are triggering events (aka triggers).

## Triggering Workflows
#### Events trigger workflows. They serve as the signal for work to start happening if a GitHub Actions workflow is present and the triggering event matches the start conditions specified in the workflow. An event can be defined in several different ways:

- A person or a process does some operation in a GitHub repository.
- A matching external trigger happens—that is, an event from outside of GitHub.
- A schedule is set up to run a workflow at particular times or intervals.
- A workflow is initiated manually, without an operation having to be done first.

#### I’ll dive into these different types more in Chapter 3 and extensively in Chapter 8, but an event triggered from an operation in a GitHub repository is probably the most common type. An example of this kind of event is a GitHub pull request. If you or a process initiates a pull request, then a corresponding pull request event is triggered. There is also a push event for code pushes. In GitHub, there are a large number of common operations you can do that can serve as triggers for a workflow.

#### There are also multiple ways to govern when workflows react to the triggers. To understand this, here’s the first piece of workflow syntax to become familiar with—the on clause. The on keyword and the lines that follow it define which types of triggers the workflow will match on and start executing. Some basic types of triggers and simple examples of the syntax for each follow:

The workflow can respond to a single event such as when a push happens:

on: push

The workflow can respond to a list (multiple events):

on: [push, pull_request]

The workflow can respond to event types with qualifiers, such as branches, tags, or file paths:

on:
  push:
    branches:
      - main 
      - 'rel/v*'
    tags:
      - v1.*
      - beta  
    paths:
      - '**.ts'

The workflow can execute on a specific schedule or interval (using standard cron syntax):

on:
  scheduled:
    - cron: '30 5,15 * * *'      

## Components
#### I’m using components here as an umbrella term (not an official one), for the major pieces that GitHub Actions defines for you to use to build and execute a workflow. For simplicity, I’ll just do a brief survey of each one to help you understand them from a higher level.

## Steps
#### Steps are the basic unit of execution you deal with when working with GitHub Actions. They consist of either invocations of a predefined action or a shell command to be run on the runner. Any shell commands are executed via a run clause. And any predefined actions are pulled in via a uses clause. The steps keyword indicates the start of a series of steps to be run sequentially.

#### The code listing that follows shows an example of three basic steps from a workflow. These steps check out a set of code, set up a go environment based on a particular version, and run the go process on a source file. In the YAML syntax, the - character indicates where a step starts. The uses clause indicates that this step invokes a predefined action. The with clause is used to specify arguments/parameters to pass to the action. And the run clause indicates a command to be run in the shell.

## Runners

#### Runners are the physical or virtual computers or containers where the code for a workflow is executed. They can be systems provided and hosted by GitHub (and run within their control), or they can be instances you set up, host, and control. In either case, the systems are configured to understand how to interact with the GitHub Actions framework. This mean they can interact with GitHub to access workflows and predefined actions, execute steps, and report outcomes.

In a workflow file, runners are defined for jobs simply via the runs-on clause. (Runners are discussed in more detail in Chapter 5.)

runs-on: ubuntu-latest

## Jobs
#### Jobs aggregate steps and define which runner to execute them on. An individual job is usually targeted towards accomplishing one particular goal of the overall workflow. An example could be a workflow that implements a CI/CD pipeline with separate jobs for building, testing, and packaging.

#### Aside from the definition of the runner, a job in a workflow is like a function or procedure in a programming language. It is made up of a series of individual commands to run and/or predefined actions to call. This is similar to how a function or procedure in a programming language is made of individual lines of code and/or calls to other functions or procedures.

#### The outcome of the job is surfaced in the GitHub Actions interfaces. Success or failure is displayed at the level of the job, not the individual steps. It’s helpful to keep this success/failure status at the job level in mind when determining how much work you want any individual job to do. It’s also helpful when considering how much detail you want to know about success or failure within the workflow execution without having to drill down.

Building on the steps and runners previously shown, the next listing shows a simple job that does the checkout and setup and performs a build:

jobs:
  build:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      - name: setup Go version'
        uses: actions/setup-go@v2
        with:
          go-version: '1.14.0'
      - run: go run helloworld.go

## Workflow
#### A workflow is like a pipeline. At a high level, it first defines the types of inputs (events) that it will respond to and under what conditions it will respond to them. This is what we talked about in the earlier section on events. The response, if the events and conditions match, is to then execute the series of jobs in the workflow, which, in turn, execute the steps for each job.

#### The overall flow is like a continuous integration process in that it responds to a particular kind of change and kicks off an automated sequence of work. The next listing shows an example of a simple workflow for processing Go programs built on the previous definitions:


1. name: Simple Go Build
2.
3. on:
4.   push:
5.     branches:
6.       - main
7.
8. jobs:
9.   build:
10.    runs-on: ubuntu-latest 
11.    steps:
12.      - uses: actions/checkout@v3
13.      - name: Setup Go version
14.        uses: actions/setup-go@v2
15.        with:
16.          go-version: '1.15.1'
17.      - run: go run hello-world.go
18.      

- Line 1: The workflow file is assigned a name.

- Line 3: This is the on identifier discussed in the section on events.

- Lines 4–6: This workflow is triggered when a push operation is done to the branch main in this GitHub repository.

- Line 8: This starts the jobs portion of the workflow.

- Line 9: There is one job in this workflow, named build.

- Line 10: This job will be executed on a runner system, hosted by GitHub, provisioned with a standard ubuntu operating system image.

- Line 11: This starts the series of steps for this job.

- Line 12: The first step is done via pulling in a predefined action. Note the way this is referenced. actions/checkout@v3 refers to the relative path after github.com, so this really says it is going to run/use the action defined at github.com/actions/checkout.

#### Also notice that this is the only line in this step—no parameters need to be passed to this action because it assumes it is checking out the source from this repository since it is in this repository.

- Line 13: The hyphen at the start of this line indicates this is the start of a second step. This line is giving the new step a name.

- Line 14: The same step is pulling in another predefined action to set up the Go environment.

- Lines 15–16: The setup-go action needs a parameter—the version of Go to use. The parameter is passed as an input to the action via a with clause.

- Line 17: This is another step, one that simply runs a command as indicated by the run keyword. The command is to execute the go run command on an example file in the repository.

## Workflow Execution
#### If you push the .github/workflows/simple-go-build.yml file and a corresponding hello-world.go file to a GitHub repository, you can see the workflow actually run right away. This is because the event condition set in the workflow (on a push to main) would match. So the workflow would be triggered and executed as soon as it is pushed.